# Text Mining for Economics and Finance HW3 - Group 9

## Andres Brito Barreiro, Hans-Christian Aarnio, Hao Yao, Steven Kingaby

## Question 1

The corpus includes 64,706 reviews of Amazon Digital Music products, from 1997 to 2014. The code below uses the LASSO, Naive Bayesian Classifier, and Multinomial Inverse Regressio models.

In [21]:
# Import the required packages
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import cross_val_score

In [27]:
# Load the dataset
ratings = pd.read_csv('Digital_Music_new.csv')
ratings

,Unnamed: 0,Rating,Review,Summary,Time
0,0,5.0,"It's hard to believe ""Memory of Trees"" came ou...",Enya's last great album,"09 12, 2006"
1,1,5.0,"A clasically-styled and introverted album, Mem...",Enya at her most elegant,"06 3, 2001"
2,2,5.0,I never thought Enya would reach the sublime h...,The best so far,"07 14, 2003"
3,3,5.0,This is the third review of an irish album I w...,Ireland produces good music.,"05 3, 2000"
4,4,4.0,"Enya, despite being a successful recording art...",4.5; music to dream to,"01 17, 2008"
...,...,...,...,...,...
64701,64701,4.0,I like the reggae sound a lot in this song. I ...,Cool song,"06 24, 2014"
64702,64702,5.0,I first heard this on Sirius and had to have i...,Great Song,"07 9, 2014"
64703,64703,5.0,"I absolutely love this song, it downloaded fin...",Five Stars,"07 13, 2014"
64704,64704,3.0,"Reggae, island beats aren't really my cup of t...",Well-crafted song,"07 9, 2014"


According to the ratings of these products, we could classify our sample into two sets, highly rated and poorly rated. To do this, we use np.where with the condition that the Rating is greater than or equal to 4 out of 5 to split the observation into two groups, highly rated sample and poorly rated sample. If the product is highly rated, the indicator is 1; otherwise, it's 0.

In [31]:
ratings['Indicator'] = np.where(ratings['Rating'] >= 4.0, 1, 0)
ratings.Time = pd.to_datetime(ratings.Time, format = '%m %d, %Y')
ratings = ratings.drop(['Unnamed: 0'], axis = 1)
ratings

,Rating,Review,Summary,Time,Indicator
0,5.0,"It's hard to believe ""Memory of Trees"" came ou...",Enya's last great album,2006-09-12,1
1,5.0,"A clasically-styled and introverted album, Mem...",Enya at her most elegant,2001-06-03,1
2,5.0,I never thought Enya would reach the sublime h...,The best so far,2003-07-14,1
3,5.0,This is the third review of an irish album I w...,Ireland produces good music.,2000-05-03,1
4,4.0,"Enya, despite being a successful recording art...",4.5; music to dream to,2008-01-17,1
...,...,...,...,...,...
64701,4.0,I like the reggae sound a lot in this song. I ...,Cool song,2014-06-24,1
64702,5.0,I first heard this on Sirius and had to have i...,Great Song,2014-07-09,1
64703,5.0,"I absolutely love this song, it downloaded fin...",Five Stars,2014-07-13,1
64704,3.0,"Reggae, island beats aren't really my cup of t...",Well-crafted song,2014-07-09,0


## Question 2

Now we continue with the usual steps of preprocessing the reviews and summary columns. First, we write a self-defined function on text preprocessing. Then, we need to collect the documents as corpus.